In [1]:
!pip install -q aicrowd-cli
%load_ext aicrowd.magic
API_KEY = '' #Please enter your API Key from [https://www.aicrowd.com/participants/me]
!aicrowd login --api-key $API_KEY
# Downloading the Dataset
!rm -rf data
!mkdir data
%aicrowd ds dl -c starship-detection -o data

!unzip data/data.zip -d /content/images >> /dev/null

     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 170 kB 45.1 MB/s 
     |████████████████████████████████| 62 kB 877 kB/s 
     |████████████████████████████████| 209 kB 55.9 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 
     |████████████████████████████████| 76 kB 5.6 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 51 kB 6.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
API Key valid
Saved API Key successfully!


data.zip:   0%|          | 0.00/34.1M [00:00<?, ?B/s]

# Importing Libraries

In [2]:
!pip install -U opencv-python

     |████████████████████████████████| 49.9 MB 38 kB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [3]:
# Image Reading & Preprocessing
from PIL import Image, ImageDraw
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Misc.
import pandas as pd
from tqdm.notebook import tqdm
import os
from natsort import natsorted
from google.colab.patches import cv2_imshow

# Image Preprocessing

In this section we are going to learn some opencv functions which can help us detecting the starship body!

In [55]:
for i in range(300):
    image = cv2.imread(f'/content/images/{i}.jpg')
    image = cv2.GaussianBlur(image, (5, 5), None)
    image = image[50:-50,50:-50]
    image = cv2.normalize(image / 255, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    canny = cv2.Canny(gray, 80, 90)
    x, y, width, height = 0, 0, 0, 0
    for j in range(len(canny)):
        if np.any(canny[j]):
            y = j
            break
    for j in range(len(canny[0])):
        if np.any(canny[:, j]):
            x = j
            break
    for j in range(len(canny)-1, 0, -1):
        if np.any(canny[j]):
            height = j - y
            break

    for j in range(len(canny[0])-1, 0, -1):
        if np.any(canny[:, j]):
            width = j - x
            break
    x += 50
    y += 50

    if np.mean(gray) < 100 and height / width > 3:
        height -= int(height * 0.35)
    elif np.mean(gray) < 100 and width / height > 3:
        width -= int(width * 0.3)
    elif np.mean(gray) < 100 and width / height > 2:
        width -= int(width * 0.15)

    image = cv2.imread(f'/content/images/{i}.jpg')
    cv2.rectangle(image, (x, y), (x+width, y+height), (0, 0, 255), 2)
    cv2_imshow(image)
    cv2_imshow(canny)

Output hidden; open in https://colab.research.google.com to view.

In [56]:
# Getting all the testing images 
test_imgs = natsorted(os.listdir("images"))
print(len(test_imgs))

10000


In [57]:
# Function to generate the bounding boxes
def gen_bounding_boxes(image):
    image = cv2.GaussianBlur(image, (5, 5), None)
    image = image[50:-50,50:-50]
    image = cv2.normalize(image / 255, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(gray, 80, 90)
    x, y, width, height = 0, 0, 0, 0
    for j in range(len(canny)):
        if np.any(canny[j]):
            y = j
            break
    for j in range(len(canny[0])):
        if np.any(canny[:, j]):
            x = j
            break
    for j in range(len(canny)-1, 0, -1):
        if np.any(canny[j]):
            height = j - y
            break

    for j in range(len(canny[0])-1, 0, -1):
        if np.any(canny[:, j]):
            width = j - x
            break
    x += 50
    y += 50

    if np.mean(gray) < 100 and height / width > 3:
        height -= int(height * 0.35)
    elif np.mean(gray) < 100 and width / height > 3:
        width -= int(width * 0.3)
    elif np.mean(gray) < 100 and width / height > 2:
        width -= int(width * 0.15)
    
    return x, y, width, height

In [58]:
bboxes = []
image_ids = []

# Ground through each test image
for img_name in tqdm(test_imgs):

  # Reading the test image
  img = cv2.imread(os.path.join("images", img_name))

  # Generating the bounding boxes
  x,y,w,h = gen_bounding_boxes(img)

  # Adding the boundong boxes and image id
  bboxes.append([x,y,w,h])
  image_ids.append(int(img_name.split(".")[0]))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [59]:
# Adding the image id and bounding boxes to a dataframe
df = pd.DataFrame({"ImageID":image_ids, "bbox":bboxes})
df = df.sort_values("ImageID").reset_index(drop=True)
df

,ImageID,bbox
0,0,"[166, 215, 20, 99]"
1,1,"[274, 229, 89, 40]"
2,2,"[247, 217, 31, 98]"
3,3,"[204, 243, 117, 32]"
4,4,"[321, 214, 36, 106]"
...,...,...
9995,9995,"[191, 240, 137, 26]"
9996,9996,"[181, 245, 70, 20]"
9997,9997,"[96, 207, 208, 80]"
9998,9998,"[157, 214, 24, 102]"


In [60]:
!rm -rf assets
!mkdir assets

df.to_csv(os.path.join("assets", "submission.csv"), index=False)

**Note : Please make sure that there should be filename `submission.csv` in `assets` folder before submitting it**

## Uploading the Results

In [ ]:
!aicrowd notebook submit -c starship-detection -a assets --no-verify